# CÁC PHƯƠNG PHÁP BOOSTING VÀ ENSEMBLE

Trong notebook này, chúng ta sẽ tìm hiểu về các phương pháp không nằm trong bộ sklearn mà thường được sử dụng để thi các cuộc thi trên Kaggle và đạt được độ chính xác cao. Cụ thể, chúng ta sẽ tập trung vào các thuật toán Gradient Boosting như ***XGBoost***, ***LightGBM*** và ***CatBoost***. Chúng ta cũng sẽ tìm hiểu về cách kết hợp chúng bằng kỹ thuật Ensemble để cải thiện hiệu suất dự đoán.

### 1. Tải dữ liệu

Trong bài này ta sẽ sữ dụng dữ liệu về ung thư vú được tải từ hàm `load_breast_cancer()` của module `sklearn.datasets`.

In [22]:
from sklearn.datasets import load_breast_cancer
import pandas as pd

# Tải dữ liệu
breast_cancer = load_breast_cancer()
X = breast_cancer.data
y = breast_cancer.target

# Tạo dataframe từ dữ liệu để dễ dàng quan sát
df = pd.DataFrame(data=X, columns=breast_cancer.feature_names)
df["target"] = y

In [23]:
# Xem qua một vài dòng dữ liệu
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [24]:
# Xem kiểu dữ liệu và số giá trị rỗng của mỗi cột
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

### 2. Tiền xử lí dữ liệu

Vì dữ liệu ở các cột là kiểu số và không có giá trị rỗng nên ta chỉ cần chuẩn hóa dữ liệu sử dụng lớp `MinMaxScaler` từ module `sklearn.preprocessing`.

In [25]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X = scaler.fit_transform(X) # đưa các giá trị về khoảng [0, 1]

Sau đó ta tiến hành chia tập dữ liệu thành 2 phần: tập train và tập test sử dụng hàm `train_test_split()` từ module `sklearn.model_selection` là xong phần chuẩn bị dữ liệu.

In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 3. Xây dựng các mô hình boosting

Chúng ta sẽ sử dụng những mô hình boosting được dùng nhiều từ những cuộc thi về machine learning trên Kaggle vì chúng có độ chính xác cao trên những dữ liệu thuộc kiểu bảng (tabular data). Những mô hình đó không có sẵn trong thư viện `sklearn` mà từ những thư viện.

**Bài tập**: Hãy import 3 mô hình `XGBClassifier`, `LGBMClassifier`, `CatBoostClassifier` có sẵn lần lượt từ các thư viện `xgboost`, `lightgbm`, `catboost` (cài đặt trước đó bằng cách `pip install xgboost lightgbm catboost`) và gán vào những biến đã được khởi tạo bên dưới sao cho phù hợp. Lưu ý: Khởi tạo mỗi mô hình với tham số ***random_state=42*** đối với 2 mô hình `XGBClassifier` và `LGBMClassifier`.

In [27]:
pip install xgboost lightgbm catboost

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
xgboost_clf = None
lgbm_clf = None
catboost_clf = None

### BEGIN SOLUTION
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

xgboost_clf = XGBClassifier(random_state=42)
lgbm_clf = LGBMClassifier(random_state=42)
catboost_clf = CatBoostClassifier(random_state=42)



### END SOLUTION

In [29]:
assert xgboost_clf is not None and lgbm_clf is not None and catboost_clf is not None

Ta tiến hành huấn luyện các mô hình trên trên tập train

In [30]:
xgboost_clf.fit(X_train, y_train)
lgbm_clf.fit(X_train, y_train)
catboost_clf.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 286, number of negative: 169
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000350 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4548
[LightGBM] [Info] Number of data points in the train set: 455, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.628571 -> initscore=0.526093
[LightGBM] [Info] Start training from score 0.526093
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

Sau khi huấn luyện thành công, ta sẽ đánh giá lần lượt từng mô hình trên tập test với metric là accuracy.

**Bài tập**: Hãy sử dụng hàm `accuracy_score` từ module `sklearn.metrics` để tính toán độ chính xác của từng mô hình trên tập test và gán nó vào các biến đã tạo bên dưới sao cho phù hợp.

In [ ]:
xgboost_clf_acc = None
lgbm_clf_acc = None
catboost_clf_acc = None

### BEGIN SOLUTION
from sklearn.metrics import accuracy_score

y_pred_xgboost = xgboost_clf.predict(X_test)
xgboost_clf_acc = accuracy_score(y_test, y_pred_xgboost) # ta có thể ép kiểu lại để thỏa ý dưới nhưng thực tế thì không cần vậy

y_pred_lgbm = lgbm_clf.predict(X_test)
lgbm_clf_acc = accuracy_score(y_test, y_pred_lgbm)

y_pred_catboost = catboost_clf.predict(X_test)
catboost_clf_acc = accuracy_score(y_test, y_pred_catboost)

# xgboost_clf_acc = np.float64(xgboost_clf_acc)
# lgbm_clf_acc = np.float64(lgbm_clf_acc)
# catboost_clf_acc = np.float64(catboost_clf_acc)
### END SOLUTION

c:\Users\admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [38]:
print(y_pred_catboost.shape)
print(type(y_pred_catboost))
print(type(y_pred_catboost[0]))

(114,)
<class 'numpy.ndarray'>
<class 'numpy.int64'>


In [ ]:
# assert str(type(xgboost_clf_acc)) == "<class 'numpy.float64'>"
# assert str(type(lgbm_clf_acc)) == "<class 'numpy.float64'>"
# assert str(type(catboost_clf_acc)) == "<class 'numpy.float64'>"
assert str(type(xgboost_clf_acc)) == "<class 'numpy.float64'>"
assert str(type(lgbm_clf_acc)) == "<class 'numpy.float64'>"
assert str(type(catboost_clf_acc)) == "<class 'numpy.float64'>"

AssertionError: 

In độ chính xác của từng mô hình, ta thấy rằng độ chính xác của chúng là khá cao so với những mô hình phân loại mà ta sử dụng trước đó.

In [39]:
print("XGBClassifier Accuracy:     ", xgboost_clf_acc)
print("LGBMClassifier Accuracy:    ", lgbm_clf_acc)
print("CatBoostClassifier Accuracy:", catboost_clf_acc)

XGBClassifier Accuracy:      0.956140350877193
LGBMClassifier Accuracy:     0.9649122807017544
CatBoostClassifier Accuracy: 0.9736842105263158


### 4. Ensemble Methods

Trong Machine Learning, ***Ensemble Methods*** là một kỹ thuật mạnh mẽ được sử dụng để cải thiện hiệu suất của các mô hình dự đoán. Thay vì sử dụng một mô hình duy nhất để thực hiện dự đoán, Ensemble Methods kết hợp kết quả từ nhiều mô hình con để đưa ra dự đoán cuối cùng. Việc này có thể giúp giảm thiểu sai số và tăng tính ổn định của dự đoán.

Các phương pháp Ensemble được sử dụng rộng rãi trong các cuộc thi trên Kaggle và các ứng dụng thực tế trong thế giới thực. Dưới đây, chúng ta sẽ tìm hiểu về hai kỹ thuật Ensemble phổ biến: ***Voting Classifier*** và ***Stacking Classifier***.

#### 4.1. Voting Classifier

***Voting Classifier*** là một phương pháp Ensemble đơn giản nhưng hiệu quả. Nó làm việc bằng cách kết hợp kết quả dự đoán từ nhiều mô hình con khác nhau. Có hai loại chính của ***Voting Classifier***:

* ***Hard Voting***: Kết quả cuối cùng là kết quả của sự bỏ phiếu đa số giữa các mô hình con. Nếu hơn một nửa các mô hình dự đoán một lớp cụ thể, lớp đó sẽ được chọn là kết quả.

* ***Soft Voting***: Kết quả cuối cùng là kết quả của sự bỏ phiếu dựa trên xác suất dự đoán của các mô hình con. Nó thường làm tốt hơn so với hard voting và phù hợp khi các mô hình con có khả năng ước tính xác suất.

**Bài tập**: Hãy import lớp `VotingClassifier` từ module `sklearn.ensemble` và tạo một mô hình ensemble kết hợp từ 3 mô hình boosting phía trên và gán vào biến `voting_model` bên dưới. Lưu ý: Khởi tạo mô hình **voting** với tham số ***voting="soft"***, sử dụng lại các mô hình boosting đã tạo.

In [40]:
voting_model = None

### BEGIN SOLUTION
from sklearn.ensemble import VotingClassifier
voting_model = VotingClassifier(
    estimators=[
        ('xgb', xgboost_clf),
        ('lgbm', lgbm_clf),
        ('catboost', catboost_clf)
    ],
    # voting='hard'  # sử dụng bỏ phiếu đa số
    voting='soft'  # sử dụng xác suất trung bình
)



### END SOLUTION

In [41]:
assert voting_model.__class__.__name__ == "VotingClassifier"
assert len(voting_model.estimators) == 3

Ta tiến hành huấn luyện và đánh giá mô hình voting thôi nào

In [42]:
from sklearn.metrics import accuracy_score

voting_model.fit(X_train, y_train)
print("Voting Model Accuracy:",
      accuracy_score(y_test, voting_model.predict(X_test)))

[LightGBM] [Info] Number of positive: 286, number of negative: 169
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000313 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4548
[LightGBM] [Info] Number of data points in the train set: 455, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.628571 -> initscore=0.526093
[LightGBM] [Info] Start training from score 0.526093
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

c:\Users\admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


<details> <summary><strong>❓ Có dùng được cho “bất kỳ model nào” không?</strong></summary>

❌ Không hoàn toàn.

Với voting='hard'

✔️ Model chỉ cần có:

fit(X, y)

predict(X)

➡️ Dùng được với:

SVM

KNN

Decision Tree

Naive Bayes

Logistic Regression

…

Với voting='soft'

❌ BẮT BUỘC tất cả model phải có:

predict_proba(X)

⚠️ Chỉ cần 1 model không có predict_proba → lỗi ngay

➡️ Ví dụ model không dùng được cho soft voting:

SVM (kernel thường, nếu không bật probability=True)

Một số model custom

</details>
<details> <summary><strong>❓ Có rủi ro gì khi dùng VotingClassifier?</strong></summary>

❌ Không cải thiện nếu các model quá giống nhau
→ Cùng thuật toán, cùng feature, cùng lỗi → voting vô nghĩa

❌ Tốn RAM và thời gian train
→ Fit nhiều model song song

❌ Không giải quyết bias hệ thống
→ Nếu tất cả model đều sai cùng kiểu → voting vẫn sai

💡 Mẹo senior

Voting hiệu quả khi model đa dạng + lỗi khác nhau

Có thể dùng trọng số:

weights = [2, 1, 1]


→ model mạnh hơn có tiếng nói lớn hơn

</details>
<details> <summary><strong>❓ Hard voting là gì?</strong></summary>

Hard voting = bỏ phiếu theo nhãn dự đoán cuối cùng của từng model

Ví dụ (binary classification)
Model	predict
XGBoost	1
LightGBM	1
CatBoost	0

→ Đếm phiếu:

class 1: 2 phiếu

class 0: 1 phiếu

➡️ Kết quả = 1

Công thức (plain-text)
y_final = mode(y_1, y_2, y_3)

</details>
<details> <summary><strong>❓ Hard voting có quan tâm độ tự tin (confidence) không?</strong></summary>

❌ Không.

Dự đoán 0.51 hay 0.99 → đều tính 1 phiếu

Không phân biệt model “rất chắc” hay “hơi đoán”

➡️ Đây là điểm yếu lớn của hard voting

</details>
<details> <summary><strong>❓ Soft voting là gì?</strong></summary>

Soft voting = trung bình xác suất dự đoán của từng class, sau đó chọn class có xác suất cao nhất.

Ví dụ
| Model    | P(class=1) | P(class=0) |
| -------- | ---------- | ---------- |
| XGBoost  | 0.90       | 0.10       |
| LightGBM | 0.55       | 0.45       |
| CatBoost | 0.40       | 0.60       |

Tính trung bình
P_avg(1) = (0.90 + 0.55 + 0.40) / 3 = 0.616
P_avg(0) = (0.10 + 0.45 + 0.60) / 3 = 0.383


➡️ Kết quả = 1

Công thức tổng quát
p_final(c) = (1/N) * Σ p_i(c)
y_final = argmax_c p_final(c)

</details>
<details> <summary><strong>❓ Hard và Soft voting có thể cho kết quả KHÁC nhau không?</strong></summary>

✔️ Có – và rất quan trọng

Ví dụ
| Model   | predict | P(class=1) |
| ------- | ------- | ---------- |
| Model A | 1       | 0.51       |
| Model B | 1       | 0.52       |
| Model C | 0       | 0.01       |

Hard voting

class 1: 2 phiếu
➡️ Kết quả = 1

Soft voting
P_avg(1) = (0.51 + 0.52 + 0.01) / 3 = 0.346
P_avg(0) = (0.49 + 0.48 + 0.99) / 3 = 0.653


➡️ Kết quả = 0

👉 Đây là lý do soft voting thường tốt hơn khi xác suất đáng tin

</details>
<details> <summary><strong>❓ Khi nào nên dùng hard, khi nào dùng soft?</strong></summary>

| Trường hợp                            | Nên dùng |
| ------------------------------------- | -------- |
| Model không có `predict_proba`        | Hard     |
| Model có xác suất tốt, được calibrate | Soft     |
| Dataset nhỏ, model yếu                | Hard     |
| Kaggle / production ML                | Soft     |



</details>

#### 4.2. Stacking Classifier

***Stacking Classifier*** là một phương pháp Ensemble phức tạp hơn so với ***Voting Classifier***. Trong Stacking, các mô hình con được chia thành hai hoặc nhiều tầng (layers). Ở tầng đầu tiên (base layer), các mô hình con được huấn luyện trên dữ liệu huấn luyện và dự đoán trên dữ liệu kiểm tra. Sau đó, dự đoán từ các mô hình con ở tầng đầu tiên được sử dụng như đầu vào cho mô hình cấp cao hơn (meta-model) ở tầng thứ hai (meta layer) để dự đoán kết quả cuối cùng.

Stacking thường cần nhiều tính toán hơn và sử dụng nhiều tài nguyên hơn so với Voting, nhưng nó có tiềm năng để tạo ra các ensemble mạnh hơn và cải thiện hiệu suất dự đoán.

Cả hai kỹ thuật này đều có thể giúp cải thiện hiệu suất dự đoán của mô hình bằng cách kết hợp sức mạnh của nhiều mô hình con, tuy nhiên, bạn cần lựa chọn một trong số họ dựa trên bộ dữ liệu cụ thể và mục tiêu của bạn.

**Bài tập**: Hãy import lớp `StackingClassifier` từ module `sklearn.ensemble` và tạo một mô hình ensemble kết hợp từ 3 mô hình boosting phía trên theo thứ tự: `XGBClassifier`, `LGBMClassifier`, `CatBoostClassifier` với mô hình cuối cùng để kếp hợp dự đoán (tham số final_estimator) là `XGBClassifier` và gán vào biến `stacking_model` bên dưới. Lưu ý: sử dụng lại các mô hình boosting đã tạo.

In [ ]:
stacking_model = None

### BEGIN SOLUTION
from sklearn.ensemble import StackingClassifier
stacking_model = StackingClassifier(
    estimators=[
        ('xgb', xgboost_clf),
        ('lgbm', lgbm_clf),
        ('catboost', catboost_clf)
    ],
    final_estimator= xgboost_clf # có thể đặt XGBClassifier(random_state=42), nhưng tái sử dụng lại mô hình trên vẫn được

)







### END SOLUTION

In [47]:
assert stacking_model.__class__.__name__ == "StackingClassifier"
assert len(stacking_model.estimators) == 3

Ta tiến hành huấn luyện và đánh giá mô hình stacking tương tự như trên.

In [48]:
stacking_model.fit(X_train, y_train)
print("Stacking Model Accuracy:",
      accuracy_score(y_test, stacking_model.predict(X_test)))

[LightGBM] [Info] Number of positive: 286, number of negative: 169
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000365 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4548
[LightGBM] [Info] Number of data points in the train set: 455, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.628571 -> initscore=0.526093
[LightGBM] [Info] Start training from score 0.526093
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

c:\Users\admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 229, number of negative: 135
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000387 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3642
[LightGBM] [Info] Number of data points in the train set: 364, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.629121 -> initscore=0.528447
[LightGBM] [Info] Start training from score 0.528447
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

c:\Users\admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


39:	learn: 0.3925744	total: 138ms	remaining: 3.3s
40:	learn: 0.3872953	total: 140ms	remaining: 3.28s
41:	learn: 0.3830265	total: 143ms	remaining: 3.27s
42:	learn: 0.3784747	total: 146ms	remaining: 3.26s
43:	learn: 0.3738488	total: 150ms	remaining: 3.25s
44:	learn: 0.3690203	total: 153ms	remaining: 3.25s
45:	learn: 0.3641140	total: 157ms	remaining: 3.26s
46:	learn: 0.3601699	total: 161ms	remaining: 3.27s
47:	learn: 0.3559901	total: 165ms	remaining: 3.27s
48:	learn: 0.3512311	total: 169ms	remaining: 3.27s
49:	learn: 0.3477627	total: 172ms	remaining: 3.27s
50:	learn: 0.3438117	total: 176ms	remaining: 3.27s
51:	learn: 0.3392836	total: 179ms	remaining: 3.26s
52:	learn: 0.3349299	total: 182ms	remaining: 3.26s
53:	learn: 0.3310153	total: 186ms	remaining: 3.26s
54:	learn: 0.3275348	total: 190ms	remaining: 3.26s
55:	learn: 0.3235115	total: 194ms	remaining: 3.27s
56:	learn: 0.3195252	total: 198ms	remaining: 3.28s
57:	learn: 0.3157281	total: 202ms	remaining: 3.29s
58:	learn: 0.3129233	total: 205m

c:\Users\admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


### 5. Kết Luận

Trong notebook này, chúng ta đã tìm hiểu về các phương pháp không nằm trong bộ sklearn như XGBoost, LightGBM và CatBoost, những mô hình thông dụng trong các cuộc thi trên Kaggle để đạt được độ chính xác cao. Chúng ta đã thực hiện các bước cơ bản từ tiền xử lý dữ liệu, huấn luyện các mô hình, và đánh giá hiệu suất trên tập kiểm tra.

Một số điểm quan trọng cần lưu ý:
- XGBoost, LightGBM và CatBoost là các mô hình mạnh mẽ có thể mang lại kết quả tốt trên nhiều loại dữ liệu khác nhau.
- Lựa chọn mô hình thường dựa trên thử nghiệm và kiến thức về từng thuật toán. XGBoost thường là một lựa chọn tốt để bắt đầu, nhưng LightGBM và CatBoost có thể mang lại kết quả tốt hơn trong một số trường hợp.
- Kỹ thuật Ensemble, như sử dụng `VotingClassifier`, có thể cải thiện độ chính xác của mô hình bằng cách kết hợp sức mạnh của các mô hình khác nhau.

Hãy nhớ rằng để thành công trên Kaggle hoặc trong bất kỳ dự án Machine Learning nào, việc thử nghiệm và tinh chỉnh là quan trọng. Hãy luôn cân nhắc và tùy chỉnh phương pháp của mình để phù hợp với bài toán cụ thể và dữ liệu của bạn.

Chúc các bạn thành công trong việc tham gia các cuộc thi trên Kaggle và trong việc áp dụng Machine Learning vào thực tế!
